# Motion Detector

This Jupyter notebook should be opened using the following commands:
~~~bash
    workon jupyter
    jupyter notebook
~~~

In [1]:
import imutils
import time
import cv2
import argparse
import os
import shutil
import pandas as pd
import sys
import glob

In [2]:
start = time.time()

In [3]:
# Uncomment the following line to test code within a Jupyter notebook
sys.argv = ['motion_detector.py', 'data/ants/VID_20190810_120737', 'VID_20190810_120737.mp4']

parser = argparse.ArgumentParser(description='Motion detector.\
    When one or more objects in motion are detected in a video frame,\
    the frame is saved as a jpg and bounding box coordinates are added to a dataframe.\
    When the video has been processed, the dataframe is saved as a CSV file\
    and the original video is optionally deleted to save storage space.')

parser.add_argument('data_dir',
                    type=str,
                    help='data directory (without trailing /) \
                    Example: /media/aubrey/9016-4EF8/ants')

parser.add_argument('video_file',
                    type=str,
                    help='video_file name \
                    Example: VID_20190810_120737.mp4')

parser.add_argument('--frame_size_factor',
                    default=4,
                    help='the frame size is shrunken by this factor before \
                    motion detection is performed to speed up processing \
                    (default=4).')

parser.add_argument('--min_area',
                    default=0.001,
                    help='minimum size of moving object in relation to area of full frame \
                    frame (default=0.001)')

parser.add_argument('--keep_video',
                    default='true',
                    help='keep the original video after processing?')
                    
args = parser.parse_args()

In [4]:
video_file_path = '{}/{}'.format(args.data_dir, args.video_file)
vs = cv2.VideoCapture(video_file_path)
original_frame_width = vs.get(3)
original_frame_height = vs.get(4)
detection_frame_width = int(original_frame_width / args.frame_size_factor)
detection_frame_height = int(original_frame_height / args.frame_size_factor)
detection_minimum_area = int(args.min_area * detection_frame_width * detection_frame_height)

# Create a directory for frames in which motion is detected
frames_dir = '{}/frames'.format(args.data_dir)
if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)

# Create an empty dataframe to store bounding boxes for objects in motion
df = pd.DataFrame(columns=['filename', 'xtl', 'ytl', 'xbr', 'ybr'])

fgbg = cv2.createBackgroundSubtractorMOG2(varThreshold=50, detectShadows=True)

# loop over the frames of the video
framenum = 0
avg = None
while True:
    # grab the current frame
    frame = vs.read()
    frame = frame[1]
    
    # If we are at the end of the file, the frame will be empty, so break out of the loop.
    if frame is None:
        break
    
    # Increment the frame count and print every 100 frames to indicate progress.
    framenum += 1
    if (framenum % 100) == 0:
        print('{} frames processed'.format((framenum)))

    # Make a copy of the original frame and resize it
    original_frame = frame
    frame = imutils.resize(frame, width=detection_frame_width)
    
    # Perform background subtraction
    thresh = fgbg.apply(frame)
    #thresh[thresh==127]=0 # Set shadows (127) to black (0)

    # Find contours on thresholded image
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    bbfound = False
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) > detection_minimum_area:
            bbfound = True

            # compute the bounding box for the contour with coordinates
            # expressed as proportion of width and height of frame
            (x, y, w, h) = cv2.boundingRect(c)
            xtl = x / detection_frame_width
            ytl = y / detection_frame_height
            xbr = (x+w) / detection_frame_width
            ybr = (y+h) / detection_frame_height
            filename = '{}/frames/{}f{:0>6}.jpg'.format(args.data_dir, args.video_file.split('.')[0], framenum)
            print('mob {} {:f} {:f} {:f} {:f}'.format(filename, xtl, ytl, xbr, ybr))
            df = df.append({'filename': filename,
                            'xtl': xtl,
                            'ytl': ytl,
                            'xbr': xbr,
                            'ybr': ybr},
                            ignore_index=True)
    # if one ot more moving objects were detected, save the original frame
    if bbfound:
        cv2.imwrite(filename, original_frame)

# Save the dataframe as a CSV        
filename ='{}/{}_bounding_boxes.csv'.format(args.data_dir, args.video_file.split('.')[0])
print('Saving bounding box data to ' + filename)
df.to_csv(filename, index=False)

vs.release()

if args.keep_video=='false':
    os.remove(video_file)
    print('{} deleted'.format(video_file))

# Display run time
print('Processing time: {} seconds'.format(int(time.time()-start)))

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000001.jpg 0.000000 0.000000 1.000000 1.000000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.100000 0.962963 0.158333 1.000000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.854167 0.696296 0.906250 1.000000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.622917 0.666667 0.679167 1.000000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.166667 0.651852 0.191667 0.759259
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.318750 0.644444 0.510417 1.000000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.133333 0.644444 0.287500 0.903704
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.685417 0.640741 1.000000 0.744444
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000002.jpg 0.000000 0.592593 0.320833 1.000000
mob data/ants/VID_20190810_1

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000045.jpg 0.954167 0.148148 1.000000 0.240741
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000045.jpg 0.281250 0.114815 0.358333 0.196296
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000046.jpg 0.952083 0.170370 1.000000 0.240741
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000046.jpg 0.310417 0.118519 0.385417 0.185185
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000047.jpg 0.935417 0.140741 1.000000 0.244444
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000047.jpg 0.341667 0.122222 0.412500 0.185185
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000048.jpg 0.931250 0.148148 1.000000 0.248148
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000048.jpg 0.368750 0.125926 0.435417 0.207407
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000049.jpg 0.927083 0.155556 1.000000 0.255556
mob data/ants/VID_20190810_1

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000096.jpg 0.304167 0.207407 0.389583 0.303704
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000097.jpg 0.272917 0.218519 0.352083 0.314815
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000098.jpg 0.239583 0.207407 0.320833 0.307407
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000099.jpg 0.222917 0.211111 0.297917 0.314815
100 frames processed
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000100.jpg 0.210417 0.188889 0.272917 0.288889
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000101.jpg 0.158333 0.188889 0.247917 0.288889
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000102.jpg 0.129167 0.188889 0.204167 0.270370
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000103.jpg 0.095833 0.155556 0.170833 0.255556
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000104.jpg 0.060417 0.148148 0.135417 0.259259
mob dat

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000195.jpg 0.900000 0.703704 0.950000 0.937037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000195.jpg 0.708333 0.177778 0.795833 0.281481
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000196.jpg 0.900000 0.703704 0.950000 0.937037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000196.jpg 0.679167 0.170370 0.766667 0.285185
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000196.jpg 0.106250 0.000000 0.168750 0.055556
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000197.jpg 0.893750 0.703704 0.952083 0.940741
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000197.jpg 0.654167 0.177778 0.737500 0.285185
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000197.jpg 0.104167 0.000000 0.168750 0.059259
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000198.jpg 0.914583 0.800000 0.947917 0.933333
mob data/ants/VID_20190810_1

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000392.jpg 0.725000 0.144444 0.797917 0.251852
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000392.jpg 0.381250 0.144444 0.462500 0.259259
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000393.jpg 0.356250 0.137037 0.439583 0.248148
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000393.jpg 0.708333 0.129630 0.789583 0.244444
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000394.jpg 0.331250 0.137037 0.416667 0.244444
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000394.jpg 0.670833 0.133333 0.750000 0.229630
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000395.jpg 0.645833 0.129630 0.739583 0.233333
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000395.jpg 0.312500 0.129630 0.387500 0.244444
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f000396.jpg 0.287500 0.122222 0.366667 0.237037
mob data/ants/VID_20190810_1

...

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001064.jpg 0.366667 0.240741 0.416667 0.325926
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001065.jpg 0.358333 0.240741 0.416667 0.329630
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001066.jpg 0.360417 0.251852 0.416667 0.325926
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001067.jpg 0.358333 0.240741 0.416667 0.325926
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001068.jpg 0.358333 0.240741 0.416667 0.325926
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001069.jpg 0.356250 0.248148 0.416667 0.325926
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001070.jpg 0.354167 0.240741 0.416667 0.329630
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001071.jpg 0.354167 0.240741 0.416667 0.329630
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001072.jpg 0.352083 0.240741 0.416667 0.337037
mob data/ants/VID_20190810_1

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001130.jpg 0.387500 0.214815 0.447917 0.325926
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001130.jpg 0.522917 0.162963 0.585417 0.270370
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001131.jpg 0.393750 0.218519 0.472917 0.318519
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001131.jpg 0.483333 0.192593 0.556250 0.277778
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001132.jpg 0.418750 0.200000 0.529167 0.318519
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001133.jpg 0.418750 0.188889 0.531250 0.303704
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001134.jpg 0.414583 0.188889 0.529167 0.303704
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001135.jpg 0.414583 0.188889 0.527083 0.303704
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001136.jpg 0.414583 0.200000 0.527083 0.300000
mob data/ants/VID_20190810_1

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001203.jpg 0.406250 0.262963 0.460417 0.340741
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001203.jpg 0.504167 0.196296 0.581250 0.277778
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001204.jpg 0.385417 0.233333 0.462500 0.337037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001204.jpg 0.520833 0.188889 0.591667 0.281481
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001205.jpg 0.395833 0.237037 0.458333 0.340741
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001205.jpg 0.535417 0.185185 0.620833 0.292593
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001206.jpg 0.381250 0.251852 0.458333 0.340741
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001206.jpg 0.558333 0.211111 0.639583 0.300000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001207.jpg 0.383333 0.255556 0.458333 0.340741
mob data/ants/VID_20190810_1

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001263.jpg 0.406250 0.266667 0.458333 0.337037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001264.jpg 0.402083 0.262963 0.458333 0.337037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001265.jpg 0.400000 0.262963 0.458333 0.337037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001266.jpg 0.395833 0.262963 0.458333 0.337037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001267.jpg 0.391667 0.259259 0.458333 0.337037
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001268.jpg 0.395833 0.240741 0.458333 0.333333
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001269.jpg 0.387500 0.233333 0.458333 0.333333
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001270.jpg 0.379167 0.240741 0.450000 0.333333
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001271.jpg 0.375000 0.248148 0.447917 0.333333
mob data/ants/VID_20190810_1

mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001344.jpg 0.189583 0.200000 0.262500 0.303704
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001345.jpg 0.156250 0.211111 0.233333 0.311111
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001346.jpg 0.141667 0.222222 0.227083 0.300000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001347.jpg 0.122917 0.214815 0.195833 0.311111
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001348.jpg 0.102083 0.211111 0.170833 0.288889
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001349.jpg 0.083333 0.196296 0.143750 0.300000
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001350.jpg 0.058333 0.162963 0.125000 0.270370
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001351.jpg 0.020833 0.166667 0.100000 0.251852
mob data/ants/VID_20190810_120737/frames/VID_20190810_120737f001352.jpg 0.014583 0.159259 0.075000 0.240741
mob data/ants/VID_20190810_1

In [5]:
# Uncomment the following line to create a pure python script called motion_detector.py
#!jupyter nbconvert --to=script 'motion_detector'